In [6]:
from dotenv import load_dotenv
_ = load_dotenv()

In [7]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


- If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [9]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [10]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [11]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4-turbo")
abot = Agent(model, [tool], system=prompt)

In [13]:
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

In [14]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ldP1KNCEwMpF4gxm6j8nxy8V'}
Back to the model!


In [15]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ldP1KNCEwMpF4gxm6j8nxy8V', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 153, 'total_tokens': 175}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_31a6c0accf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-00fa3d57-8866-432e-bf84-f28890d938b3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ldP1KNCEwMpF4gxm6j8nxy8V'}], usage_metadata={'input_tokens': 153, 'output_tokens': 22, 'total_tokens': 175}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'la

In [16]:
result['messages'][-1].content

'The current weather in San Francisco is misty with a temperature of 50°F (10°C). The wind is blowing from the north at about 2.2 mph, and the humidity is at 100%. The conditions also include a visibility of about 3.0 miles.'

In [17]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? What is the GDP of state where the winning team is located?" 
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_i6jqTy9DZNqGHGGZNLkWcQ0b'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Kansas City, Missouri 2023'}, 'id': 'call_9nw3yMrfZrpzsObpYRemmtYD'}
Back to the model!


In [18]:
result['messages'][-1].content

'The Kansas City Chiefs won the Super Bowl in 2024. They defeated the San Francisco 49ers in a thrilling overtime victory.\n\nThe Gross Domestic Product (GDP) of Kansas City, Missouri, where the Chiefs are located, is not directly provided in recent data for 2023. However, historical data might be extrapolated to estimate the GDP for 2023, though specific figures from trustworthy sources are needed for an accurate number.'

> Interresting side note. If you look through the information returned by the search engine, `print(result['messages'])`, you may find the search results do not mention the state that the Kansas City Chiefs are located in. This information is then drawn from the LLM intrinsic knowledge. This is not completely trivial as Kansas City is in both Missouri and Kansas. 